In [17]:
import pandas as pd

In [25]:
import os
import openai
import json

df = pd.read_json('data/metadata/top_rated_1000.json')

# try to load the api key from file
secrets_filename = os.path.join(os.path.abspath(
    os.path.dirname('eda.ipynb')), 'ai_posters/secret2.txt')
try:
    with open(secrets_filename, 'r') as f:
        api_key = f.read().strip()
        openai.api_key = api_key
except Exception as e:
    print(e)


offset = 0
ctr = 0
limit = 20

missing_desc = df[df['description'].isnull()]
for idx, row in missing_desc.iterrows():
    if ctr + offset >= limit:
        break
    ctr += 1
    movie_json_str = row.to_json(orient='records')
    # print(movie_json_str)
    prompt = """
    for the movie and movie_id, describe the plot in 50 words or less without spoilers. 
    Swap all names used for similar ones including the title of the movie, any character names,
    and any location names. do not mention the rating metadata, only focus on the movie's 
    plot and characters. Make sure your ouput is valid json with opening and closing brackets
    ###json begins
    """
    prompt += movie_json_str
    prompt += '\n###json ends'

    # print(prompt)

    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": """You are a helpful assistant that
                transforms movie titles and descriptions. Output is json with 
                two fields: substituted_name, and description. Ensure that 
                the fields are correctly formatted as .json. Only return one object."""},
                  {"role": "user", "content": prompt}
                  ],
        temperature=0.5,
        max_tokens=1024,
        n=1,
        stop=None,
        timeout=30,
        presence_penalty=0.5,
        frequency_penalty=0.5
    )
    # print(res)
    choices = res['choices']
    if len(choices) != 1:
        print('1 choice was not returned')
    for choice in choices:
        desc = json.loads(str(choice['message']['content']))
        if type(desc) == type(list()):
            desc = desc[0]
        # print(desc)
        sub_name = desc['substituted_name']
        description = desc['description']
        print('sub name of ', sub_name)
        df.loc[df.name == row['name'], 'substituted_name'] = sub_name
        df.loc[df.name == row['name'], 'description'] = description
        print(df.loc[idx]['substituted_name'])

df.to_json('data/metadata/top_rated_1000.json', orient='records')


sub name of  Anxiety and Dislike in Los Angeles (1998)
Anxiety and Dislike in Los Angeles (1998)
sub name of  Rival Gangs of New York
Rival Gangs of New York
sub name of  Switched
Switched
sub name of  Undercover Soft (1996)
Undercover Soft (1996)
sub name of  Flutter Effect, The (2004)
Flutter Effect, The (2004)


JSONDecodeError: Expecting property name enclosed in double quotes: line 4 column 1 (char 311)